In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

In [2]:
#Reading the csv file
df = pd.read_csv('NER_CRF.csv', encoding = "ISO-8859-1")

In [3]:
df.head(10)


,Sentence #,Word,POS,Tag
0,Sentence: 1,couple,NN,Gender_actor
1,NaN,22-year-old,CD,Age_survivor
2,NaN,woman,NN,Gender_survivor
3,NaN,raped,VBN,Violence_Verb
4,NaN,maid,NNP,Relationship
5,NaN,accommodation,NN,Place
6,NaN,An,DT,Others
7,NaN,India,NNP,Others
8,NaN,was,VBD,Others
9,NaN,arrested,VBN,Others


In [4]:
df.describe()


,Sentence #,Word,POS,Tag
count,602,36070,36071,36037
unique,602,6468,46,14
top,Sentence: 255,the,NN,Others
freq,1,1998,6108,33567


In [5]:
#Displaying the unique Tags
df['Tag'].unique()

array(['Gender_actor', 'Age_survivor', 'Gender_survivor', 'Violence_Verb',
       'Relationship', 'Place', 'Others', 'Circumstances', 'Age_actor',
       'Medium_Violence', 'Time', 'Challenged_actor', nan,
       'Challenged_survivor', 'Orientation_survivor'], dtype=object)

In [6]:
#Checking null values, if any.
df.isnull().sum()

Sentence #    35469
Word              1
POS               0
Tag              34
dtype: int64

In [7]:
df = df.fillna(method = 'ffill')

In [8]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [9]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'couple  22-year-old woman  raped   maid accommodation An India was arrested one week after a accused them of rape Reports indicate that the accused allegedly the house at their rented'

In [10]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)

[('couple ', 'NN', 'Gender_actor'), ('22-year-old', 'CD', 'Age_survivor'), ('woman ', 'NN', 'Gender_survivor'), ('raped ', 'VBN', 'Violence_Verb'), (' maid', 'NNP', 'Relationship'), ('accommodation', 'NN', 'Place'), ('An', 'DT', 'Others'), ('India', 'NNP', 'Others'), ('was', 'VBD', 'Others'), ('arrested', 'VBN', 'Others'), ('one', 'CD', 'Others'), ('week', 'NN', 'Others'), ('after', 'IN', 'Others'), ('a', 'DT', 'Others'), ('accused', 'VBD', 'Others'), ('them', 'PRP', 'Others'), ('of', 'IN', 'Others'), ('rape', 'NN', 'Others'), ('Reports', 'NNS', 'Others'), ('indicate', 'VBP', 'Others'), ('that', 'IN', 'Others'), ('the', 'DT', 'Others'), ('accused', 'VBN', 'Others'), ('allegedly', 'RB', 'Others'), ('the', 'DT', 'Others'), ('house', 'NN', 'Others'), ('at', 'IN', 'Others'), ('their', 'PRP$', 'Others'), ('rented', 'VBN', 'Others')]


In [11]:
sentences = getter.sentences

In [12]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [13]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [15]:
crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [16]:
#Predicting on the test set.
y_pred = crf.predict(X_test)

In [17]:
print(y_pred)

[['Violence_Verb', 'Gender_survivor', 'Age_actor', 'Age_survivor', 'Gender_survivor', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others', 'Others'], ['Gender_survivor', 'Violence_Verb', 'Other

In [18]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.9703182733792345


In [19]:

report = flat_classification_report(y_test, y_pred)
print(report)

                      precision    recall  f1-score   support

           Age_actor       0.70      0.50      0.58        38
        Age_survivor       0.60      0.55      0.57        44
 Challenged_survivor       0.00      0.00      0.00         2
       Circumstances       0.68      0.38      0.49        45
        Gender_actor       0.90      0.73      0.81        78
     Gender_survivor       0.80      0.64      0.71        88
     Medium_Violence       1.00      0.55      0.71        11
Orientation_survivor       0.00      0.00      0.00         1
              Others       0.98      1.00      0.99      7645
               Place       0.58      0.27      0.37        26
        Relationship       0.85      0.56      0.67        61
                Time       0.50      0.09      0.15        11
       Violence_Verb       0.78      0.76      0.77       136

            accuracy                           0.97      8186
           macro avg       0.64      0.46      0.52      8186
      

/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
